# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Word2Vec
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
from numba import jit
import warnings
from IPython.display import clear_output
os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## TaggedDocument

In [5]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Load Data

#### Path

In [6]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentsPath = '/Volumes/disk1/data/daum_Comments/'
    naverCommentsPath = '/Volumes/disk1/data/naver_Comments/'
    outcomeDaumCommentsPath = '/Volumes/disk1/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = '/Volumes/disk1/outcome_comments_for_naver/'
    outcome_predata = '/Volumes/disk1/pre_data_for_comments/'
    outcome_tagged_data = '/Volumes/disk1/pre_data_for_comments2/'
    outcome_vectorized_data = '/Volumes/disk1/pre_data_for_comments3/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentsPath = 'd:/data/daum_Comments/'
    naverCommentsPath = 'd:/data/naver_Comments/'
    outcomeDaumCommentsPath = 'd:/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = 'd:/outcome_comments_for_naver/'
    outcome_predata = 'd:/pre_data_for_comments/'
    outcome_tagged_data = 'd:/pre_data_for_comments2/'
    outcome_vectorized_data = 'd:/pre_data_for_comments3/'

#### News

In [7]:
os.listdir(news_senti_outcome)

['naver_news_sentiment_analysis.csv', 'daum_news_sentiment_analysis.csv']

In [8]:
# Naver
naverData = pd.read_csv(os.path.join(news_senti_outcome, 'naver_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
naverData['site'] = ['Naver'] * naverData.shape[0]
reNaverData = naverData[naverData.number_of_crawled_comment != 0]
print (reNaverData.shape)
reNaverData.head()

(15015, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1465,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","['외무성', '핵전쟁', '대변인']","{'미국', '조선반도', '핵전쟁', '북한', '중앙', '도화선', '고위',...",86.0,22.0,negative,Naver
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2062,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"['예산', '예결위', 'soc']","{'호남', '예산안', '의원', '정부안', '증액', '지역구', '국민의당'}",46.0,62.0,positive,Naver
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,536,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"['최경환', '구속영장', '국가정보원']","{'구속영장 청구', '조사', '의원', '정기국회', '국정원장', '검찰', ...",77.0,31.0,negative,Naver
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,133,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"['이슈 · 최재형 감사원장 내정', '감사원장', '최재형', '감사원']","{'공직 사회', '법관', '생활', '감사원장', '지명', '후보자'}",39.0,69.0,positive,Naver
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,932,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"['김정은', 'b-1b', '한반도']","{'공장', '양강도', '사무차장', '삼지연', '시찰', '훈련', '접경',...",72.0,36.0,negative,Naver


In [9]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
reDaumData = daumData[daumData.number_of_crawled_comment != 0]
print (reDaumData.shape)
reDaumData.head()

(9369, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,903,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","['부작용', '비아그라', '아일랜드']","{'건강', '부작용', '공장', '남성들', '지역', '세보 효과', '연기'}",50.0,58.0,positive,daum
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"['커피', '낮잠', '음식물']","{'건강', '커피', '자세', '치아', '철분', '디스크', '점심 식사',...",81.0,27.0,negative,daum
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,807,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"['살인혐의', '철원', '검찰송치']","{'앙심', '남편', '성관계', '주변', '진술', '지인', '범행', '아...",99.0,9.0,negative,daum
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,368,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","['신영자', '갑질', '롯데백화점']","{'네이처리퍼블릭', '롯데', '징역', '유통업체', '신영자 이사장', '검찰...",77.0,31.0,negative,daum
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,367,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"['공범', '살인', '과자']","{'폭행', '혐의', '과자', '범행', '쪽지', '남자친구', '경찰', '...",102.0,6.0,negative,daum


### 뉴스별 댓글에 따른 분석 진행
* Doc2Vec의 첫번째 jupyter notebook으로 진행을 했을때 죽었으므로 fastText도 죽을 것으로 짐작되어 코드를 수정함

In [10]:
os.listdir(outcome_predata)

['predata_for_daum_news_comment.csv',
 'predata_for_naver_news_comment.csv',
 'filtered_predata_for_naver_news_comment.csv',
 'filtered_predata_for_daum_news_comment.csv']

In [11]:
%%time
predata_naver = outcome_predata +  'filtered_predata_for_naver_news_comment.csv'
dfNaver = pd.read_csv(predata_naver, header = 0, index_col = 0, encoding = 'utf-8')
print (dfNaver.shape)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(11888645, 8)
CPU times: user 50.9 s, sys: 9.08 s, total: 1min
Wall time: 1min 6s


In [12]:
#dfNaver = dfNaver[dfNaver._id == '5a29c445588c132954d1973a']

In [13]:
extDfNaver = dfNaver.loc[:,['_id', 'category', 'date', 'rank', 'site', '공감', '비공감']]

In [14]:
%%time
predata_daum = outcome_predata +  'filtered_predata_for_daum_news_comment.csv'
dfDaum = pd.read_csv(predata_daum, header = 0, index_col = 0, encoding = 'utf-8')
print (dfDaum.shape)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4924599, 8)
CPU times: user 20.1 s, sys: 3.06 s, total: 23.2 s
Wall time: 32.7 s


In [15]:
#dfDaum = dfDaum[dfDaum._id =='5a2a61bf588c13481c229d1e']

In [16]:
extDfDaum = dfDaum.loc[:,['_id', 'category', 'date', 'rank', 'site', '공감', '비공감']]

## Word2Vec Model

In [17]:
def PipeLineForSentimentUsingWord2Vec(extCommentsDf, token, tfidf, size, model, classifierDict, outcomeFileName):
    import pandas as pd
    from glob import glob
    import Basic_Module as bm
    import os
    wv, inferVector_by_model = bm.Make_Pre_Data_For_DAUM(model, tfidf, size, token)
    predict_Outcome = dict(map(lambda x: bm.PredictSentiment(inferVector_by_model, x, classifierDict[x]), classifierDict))
    predict_Outcome = pd.DataFrame.from_dict(predict_Outcome)
    predict_Outcome = extCommentsDf.merge(predict_Outcome,
                                   left_index = True, right_index = True)
    predict_Outcome.to_csv(outcomeFileName, index=None, encoding='utf-8')
    

##### Load Model

In [18]:
taggerName_ct = 'ct'
print ( '{}'.format(taggerName_ct))
model1_ct = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model1_ct_Name = bm.Return_ModelName('word2vec', model1_ct, taggerName_ct)

model2_ct = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model2_ct_Name = bm.Return_ModelName('word2vec', model2_ct, taggerName_ct)

model3_ct = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')
model3_ct_Name = bm.Return_ModelName('word2vec', model3_ct, taggerName_ct)

ct


In [19]:
taggerName_mecab = 'mecab'
print ( '{}'.format(taggerName_mecab))
model1_mecab = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-mecab.model')
model1_mecab_Name = bm.Return_ModelName('word2vec', model1_mecab, taggerName_mecab)

model2_mecab = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-mecab.model')
model2_mecab_Name = bm.Return_ModelName('word2vec', model2_mecab, taggerName_mecab)

model3_mecab = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-mecab.model')
model3_mecab_Name = bm.Return_ModelName('word2vec', model3_mecab, taggerName_mecab)

mecab


#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [20]:
trainName_ct = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_ct.pickled'
train_ct = pickle.load(open(trainName_ct, 'rb'))
tfidf_ct = bm.Build_tfidf(train_ct)
del train_ct

100%|██████████| 442359/442359 [00:00<00:00, 1383045.05it/s]


(442359, 159113)
vocab size : 159113


In [21]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled'
train_mecab = pickle.load(open(trainName, 'rb'))
tfidf_mecab = bm.Build_tfidf(train_mecab)
del train_mecab

100%|██████████| 442359/442359 [00:00<00:00, 864549.93it/s] 


(442359, 161590)
vocab size : 161590


##### Load Classifier

In [22]:
classifier_by_model1_ct = glob(classifierPath+'*'+model1_ct_Name)
load_Classifier_by_model1_ct_Dict = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model1_ct))

classifier_by_model2_ct = glob(classifierPath+'*'+model2_ct_Name)
load_Classifier_by_model2_ct_Dict = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model2_ct))

classifier_by_model3_ct = glob(classifierPath+'*'+model3_ct_Name)
load_Classifier_by_model3_ct_Dict = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model3_ct))

In [24]:
classifier_by_model1_mecab = glob(classifierPath+'*'+model1_mecab_Name)
load_Classifier_by_model1_mecab_Dict = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model1_mecab))

classifier_by_model2_mecab = glob(classifierPath+'*'+model2_mecab_Name)
load_Classifier_by_model2_mecab_Dict = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model2_mecab))

classifier_by_model3_mecab = glob(classifierPath+'*'+model3_mecab_Name)
load_Classifier_by_model3_mecab_Dict = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model3_mecab))

In [25]:
warnings.filterwarnings('ignore')
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')
for idx in reNaverData.index:
    clear_output()
    data = reNaverData.loc[idx]
    newsId = data.name
    print (newsId)
    siteInfo = data.site
    if os.path.isdir(os.path.join(outcomeNaverCommentsPath, newsId)):
        outcomePath = os.path.join(outcomeNaverCommentsPath, newsId)
    else:
        os.mkdir(os.path.join(outcomeNaverCommentsPath, newsId))
        outcomePath = os.path.join(outcomeNaverCommentsPath, newsId)
    commentsFile = naverCommentsPath + newsId + '.csv'
    commentsDf = pd.read_csv(commentsFile, encoding='utf-8', header=0)
    commentsDf.dropna(inplace = True)
    extCommentsDf = commentsDf.loc[:,['category', 'comments', 'date', 'rank', 'site', '공감', '비공감']]
    # twitter
    tagged_naver_ct = bm.MakeTaggedData_For_Comments(commentsDf, TaggedDocument, ct, stopwords)
    
    outcomeFileName1_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model1_ct_Name))
    if not os.path.isfile(outcomeFileName1_ct):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_naver_ct, tfidf_ct, 1000, model1_ct, load_Classifier_by_model1_ct_Dict, outcomeFileName1_ct)
    outcomeFileName2_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model2_ct_Name))
    if not os.path.isfile(outcomeFileName2_ct):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_naver_ct, tfidf_ct, 1000, model2_ct, load_Classifier_by_model2_ct_Dict, outcomeFileName2_ct)
    outcomeFileName3_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model3_ct_Name))
    if not os.path.isfile(outcomeFileName3_ct):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_naver_ct, tfidf_ct, 1000, model3_ct, load_Classifier_by_model3_ct_Dict, outcomeFileName3_ct)
    del tagged_naver_ct
    # Mecab
    tagged_naver_mecab = bm.MakeTaggedData_For_Comments(commentsDf, TaggedDocument, mecab, stopwords)
    
    outcomeFileName1_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model1_mecab_Name))
    if not os.path.isfile(outcomeFileName1_mecab):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_naver_mecab, tfidf_mecab, 1000, model1_mecab, load_Classifier_by_model1_mecab_Dict, outcomeFileName1_mecab)
    outcomeFileName2_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model2_mecab_Name))
    if not os.path.isfile(outcomeFileName2_mecab):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_naver_mecab, tfidf_mecab, 1000, model2_mecab, load_Classifier_by_model2_mecab_Dict, outcomeFileName2_mecab)
    outcomeFileName3_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model3_mecab_Name))
    if not os.path.isfile(outcomeFileName3_mecab):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_naver_mecab, tfidf_mecab, 1000, model3_mecab, load_Classifier_by_model3_mecab_Dict, outcomeFileName3_mecab)
    del tagged_naver_mecab
    del commentsDf, extCommentsDf

  0%|          | 0/1465 [00:00<?, ?it/s]

5a29c445588c132954d1973a


 11%|█         | 17243/162640 [00:00<00:01, 85816.82it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:02<00:00, 65992.06it/s]
4it [00:00, 32.76it/s]

...

100%|██████████| 165361/165361 [00:01<00:00, 136229.76it/s]
10it [00:00, 98.83it/s]

running time : 0:00:01.217121
Vectorizing Data


1465it [00:02, 586.03it/s]
1465it [00:00, 704662.31it/s]


scaling Data
total running time : 0:00:03.750636


1465it [00:00, 572714.64it/s]
1465it [00:00, 737388.14it/s]


In [26]:
warnings.filterwarnings('ignore')
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')
for idx in reDaumData.index:
    clear_output()
    data = reDaumData.loc[idx]
    newsId = data.name
    print (newsId)
    siteInfo = data.site
    if os.path.isdir(os.path.join(outcomeDaumCommentsPath, newsId)):
        outcomePath = os.path.join(outcomeDaumCommentsPath, newsId)
    else:
        os.mkdir(os.path.join(outcomeDaumCommentsPath, newsId))
        outcomePath = os.path.join(outcomeDaumCommentsPath, newsId)
    commentsFile = daumCommentsPath + newsId + '.csv'
    commentsDf = pd.read_csv(commentsFile, encoding='utf-8', header=0)
    commentsDf.dropna(inplace = True)
    extCommentsDf = commentsDf.loc[:,['category', 'comments', 'date', 'rank', 'site', '공감', '비공감']]
    # twitter
    tagged_daum_ct = bm.MakeTaggedData_For_Comments(commentsDf, TaggedDocument, ct, stopwords)
    
    outcomeFileName1_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model1_ct_Name))
    if not os.path.isfile(outcomeFileName1_ct):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_daum_ct, tfidf_ct, 1000, model1_ct, load_Classifier_by_model1_ct_Dict, outcomeFileName1_ct)
    outcomeFileName2_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model2_ct_Name))
    if not os.path.isfile(outcomeFileName2_ct):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_daum_ct, tfidf_ct, 1000, model2_ct, load_Classifier_by_model2_ct_Dict, outcomeFileName2_ct)
    outcomeFileName3_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model3_ct_Name))
    if not os.path.isfile(outcomeFileName3_ct):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_daum_ct, tfidf_ct, 1000, model3_ct, load_Classifier_by_model3_ct_Dict, outcomeFileName3_ct)
    del tagged_daum_ct
    # Mecab
    tagged_daum_mecab = bm.MakeTaggedData_For_Comments(commentsDf, TaggedDocument, mecab, stopwords)
    
    outcomeFileName1_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model1_mecab_Name))
    if not os.path.isfile(outcomeFileName1_mecab):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_daum_mecab, tfidf_mecab, 1000, model1_mecab, load_Classifier_by_model1_mecab_Dict, outcomeFileName1_mecab)
    outcomeFileName2_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model2_mecab_Name))
    if not os.path.isfile(outcomeFileName2_mecab):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_daum_mecab, tfidf_mecab, 1000, model2_mecab, load_Classifier_by_model2_mecab_Dict, outcomeFileName2_mecab)
    outcomeFileName3_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, model3_mecab_Name))
    if not os.path.isfile(outcomeFileName3_mecab):
        PipeLineForSentimentUsingWord2Vec(extCommentsDf, tagged_daum_mecab, tfidf_mecab, 1000, model3_mecab, load_Classifier_by_model3_mecab_Dict, outcomeFileName3_mecab)
    del tagged_daum_mecab
    del commentsDf, extCommentsDf

  0%|          | 1/903 [00:00<02:26,  6.17it/s]

5a2a61bf588c13481c229d1e


 19%|█▉        | 31221/162640 [00:00<00:00, 156087.84it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 147443.14it/s]
32it [00:00, 286.92it/s]

...

100%|██████████| 165361/165361 [00:01<00:00, 130695.98it/s]
160it [00:00, 1578.11it/s]

running time : 0:00:01.267492
Vectorizing Data


903it [00:00, 1608.46it/s]
903it [00:00, 560855.40it/s]
903it [00:00, 373922.06it/s]


scaling Data
total running time : 0:00:01.854748


903it [00:00, 504456.12it/s]
